# Example Python Docker Healthcheck scripts
## Based on environments with Status of 'Active - Auto' and Techs and Specs defined

The scripts look for Environments with:
- Status set to 'Active - Auto'
- Host = Automation and Layer = Application with the values set to docker containers

It relies on a credentials.cfg file withthe following content to work
> {
>   "urls":{
>     "authUrl":"https://ukoauth.plutora.com/",
>     "baseUrl":"https://ukapi.plutora.com/"
>   },
>   "credentials": {
>     "client_id":"xxxxxxxxxx",
>     "client_secret":"xxxxxxxx",
>     "username":"user email",
>     "password":"user password*"
>   }
> }

##The Initialization

In [ ]:
import docker
import json
import requests
from typing import Optional, Dict, Any
with open("credentials.cfg") as data_file:
	data = json.load(data_file)

authUrl = data["urls"]["authUrl"]
baseUrl = data["urls"]["baseUrl"]
client_id = data["credentials"]["client_id"]
client_secret = data["credentials"]["client_secret"]
username = data["credentials"]["username"]
password = data["credentials"]["password"]

#Get the Plutora Access Token function

In [15]:
def getAccessToken():
	# Returns access_token
	#
	# authUrl:
	#	https://usoauth.plutora.com/
	# client_id & client_secret
	#	Values generated by Plutora platform, see Customizations>API
	# username & password
	#	User credentials used to access the Plutora platform
	#
	# If successful, Returns: "" or JSON-formatted response
	# On failure, Returns: response text
	#
	headers = {}
	headers['content-type'] = "application/x-www-form-urlencoded"
	headers['cache-control'] = "no-cache"
	url = authUrl + "oauth/token"
	payload  = "client_id=" + client_id + '&'
	payload += "client_secret=" + client_secret + '&'
	payload += "grant_type=" + 'password&'
	payload += "username=" + username.replace('@','%40') + '&'
	payload += "password=" + password + '&'
	response = requests.request("POST", url, data=payload, headers=headers)
	if not response.ok:
		print ("Access token: ")
		print(response.text)
		exit()
	access_token = response.json()['access_token']
	return access_token
	# ------End of def getAccessToken()-----------

# Get the environments

In this case we use 

> `environments = get_plutora_environments_with_custom_filter(bearer_token, '`status` = `Active - Auto`', base_url)`

to get the environments in a state of **Active - Auto**

In [25]:
def get_plutora_environments_with_custom_filter(bearer_token: str, filter_expression: str, 
                                               base_url: str = "https://ukapi.plutora.com") -> Optional[Dict[Any, Any]]:
    """
    Retrieve environments from Plutora API with a custom filter.
    
    Args:
        bearer_token (str): The bearer token for authentication
        filter_expression (str): Custom filter expression (e.g., "`status` = `Active - Auto`")
        base_url (str): The base URL for the Plutora API (default: https://ukapi.plutora.com)
    
    Returns:
        dict: JSON response from the API if successful, None if failed
    
    Raises:
        requests.exceptions.RequestException: If the API request fails
    """
    
    url = f"{base_url}/environments"
    
    headers = {
        'accept': 'application/json',
        'Plutora-Info': 'script.name=swagger',
        'Authorization': f'bearer {bearer_token}'
    }
    
    params = {
        'filter': filter_expression
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"Error making request to Plutora API: {e}")
        return None

# get the detailed environment information based on id

example call
`> env_detail = get_environment_details(env_id, bearer_token, base_url)`

In [28]:
def get_environment_details(environment_id: str, bearer_token: str, base_url: str = "https://ukapi.plutora.com") -> Optional[Dict[Any, Any]]:
    """
    Get detailed information for a specific environment including hosts and layers.
    
    Args:
        environment_id (str): The ID of the environment to retrieve
        bearer_token (str): The bearer token for authentication
        base_url (str): The base URL for the Plutora API
    
    Returns:
        dict: Detailed environment information including hosts and layers
    """
    url = f"{base_url}/environments/{environment_id}"
    
    headers = {
        'accept': 'application/json',
        'Plutora-Info': 'script.name=swagger',
        'Authorization': f'bearer {bearer_token}'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"Error getting environment details for {environment_id}: {e}")
        return None

#     Extract version information for all layers in the "Automation" host.


In [29]:
def get_automation_host_versions(environment_detail: Dict[Any, Any]) -> Dict[str, str]:
    """
    Extract version information for all layers in the "Automation" host.
    
    Args:
        environment_detail (dict): Detailed environment information from the API
    
    Returns:
        dict: Dictionary mapping component names to their versions in the Automation host
    """
    automation_versions = {}
    
    # Get hosts from the environment
    hosts = environment_detail.get('hosts', [])
    
    # Find the Automation host
    for host in hosts:
        if host.get('name') == 'Automation':
            # Extract versions from all layers in this host
            layers = host.get('layers', [])
            for layer in layers:
                component_name = layer.get('componentName', 'Unknown')
                version = layer.get('version', 'Unknown')
                automation_versions[component_name] = version
            break
    
    return automation_versions

# main function to return all layers by env id

In [37]:
def get_all_environment_automation_versions(bearer_token: str, base_url: str = "https://ukapi.plutora.com") -> Dict[str, Dict[str, str]]:
    """
    Get automation host versions for all active environments.
    
    Args:
        bearer_token (str): The bearer token for authentication
        base_url (str): The base URL for the Plutora API
    
    Returns:
        dict: Dictionary mapping environment names to their automation host versions
    """
    # First get the list of environments
    environments = get_plutora_environments_with_custom_filter(bearer_token, '`status` = `Active - Auto`', base_url)
    
    if not environments:
        print("Failed to retrieve environments list")
        return {}
    
    all_versions = {}
    
    # For each environment, get detailed info and extract automation versions
    for env in environments:
        env_id = env.get('id')
        env_name = env.get('name', 'Unknown')
        
        if env_id:
            print(f"Processing environment: {env_name}")
            
            # Get detailed environment info
            env_detail = get_environment_details(env_id, bearer_token, base_url)
            
            if env_detail:
                # Extract automation host versions
                automation_versions = get_automation_host_versions(env_detail)
                
                if automation_versions:
                    all_versions[env_id] = automation_versions
                else:
                    print(f"  No Automation host found in {env_name}")
            else:
                print(f"  Failed to get details for {env_name}")
    
    return all_versions

# check to see whether a docker container is running

In [50]:
def is_container_running(container_name_or_id: str) -> bool:
    """
    Checks if a Docker container is running using the Docker SDK.

    Args:
        container_name_or_id: The name or ID of the container.

    Returns:
        True if the container is running, False otherwise.
    """
    from docker.errors import NotFound, APIError, DockerException

    try:
        # Connect to the Docker daemon
        client = docker.from_env()
        
        # Get the container object
        container = client.containers.get(container_name_or_id)
        
        # Check the container's status
        # The state object contains detailed information
        # container.status will be one of: 'created', 'restarting', 'running', 
        # 'removing', 'paused', 'exited', or 'dead'
        if container.status == 'running':
            return True
        else:
            print(f"Container '{container_name_or_id}' found but its status is '{container.status}'.")
            return False

    except NotFound:
        # The container does not exist
        print(f"Container '{container_name_or_id}' not found.")
        return False
    except APIError as e:
        # Problem communicating with the Docker API
        print(f"Docker API error: {e}")
        return False
    except DockerException as e:
        # Other Docker-related errors (e.g., connection issues)
        print(f"Docker error: {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors
        print(f"Unexpected error: {e}")
        return False


# update Plutora healthcheck for an environment id

In [55]:
def send_environment_health_check(env_id: str, health: int, bearer_token: str, 
                                test_name: str = "docker check", 
                                base_url: str = "https://ukapi.plutora.com") -> bool:
    """
    Send environment health check to Plutora API.
    
    Args:
        env_id (str): Environment ID
        health (int): Health status (1 = healthy, 2 = unhealthy)
        bearer_token (str): Bearer token for authentication
        test_name (str): Name of the test (default: "docker check")
        base_url (str): Base URL for the API
    
    Returns:
        bool: True if successful, False otherwise
    """
    url = f"{base_url}/environmentHealthCheck"
    
    headers = {
        'accept': 'application/json',
        'Plutora-Info': 'script.name=swagger',
        'Authorization': f'bearer {bearer_token}'
    }
    
    # Prepare form data
    files = {
        'environmentId': (None, env_id),
        'health': (None, str(health)),
        'testName': (None, test_name)
    }
    
    try:
        response = requests.post(url, headers=headers, files=files)
        response.raise_for_status()
        
        print(f"Successfully sent health check for environment {env_id}: health={health}")
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"Error sending health check for environment {env_id}: {e}")
        return False

# Test the functions

In [66]:
access_token=getAccessToken()
automation_versions = get_all_environment_automation_versions(access_token)
i = 0
for env_id, versions in automation_versions.items():
    for component, container in versions.items():
        if not is_container_running(container):
            i += 1
        print(f"{env_id}  -  {component}  -  {container}  [{i}]")
    if i >= 1:
        response = send_environment_health_check(env_id, 2, access_token)
    else:
        response = send_environment_health_check(env_id, 1, access_token)
    
    i = 0
    print("=" * 50)

Processing environment: Mortgage_SIT
Processing environment: Mortgage_UAT
c2c94a3c-3df0-ef11-abc0-06b344cd22ba  -  Docker1  -  qdrant  [0]
c2c94a3c-3df0-ef11-abc0-06b344cd22ba  -  Docker2  -  redis  [0]
Successfully sent health check for environment c2c94a3c-3df0-ef11-abc0-06b344cd22ba: health=1
04ca4a3c-3df0-ef11-abc0-06b344cd22ba  -  Docker  -  n8n  [0]
Successfully sent health check for environment 04ca4a3c-3df0-ef11-abc0-06b344cd22ba: health=1
